In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [ ]:
# Load image
image_path = "../data/raw/my_dogs.jpg"

if not Path(image_path).exists():
    print("⚠️  Using sample image")
    image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
else:
    image = cv2.imread(image_path)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

print(f"Image loaded: {image.shape}")

## 2.1 Gradient-Based Edge Detection

Edges occur where pixel intensity changes rapidly

In [ ]:
# Sobel edge detection (gradients in X and Y)
sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
sobel_combined = np.sqrt(sobelx**2 + sobely**2)
sobel_combined = np.uint8(sobel_combined / sobel_combined.max() * 255)

# Laplacian (second derivative)
laplacian = cv2.Laplacian(gray, cv2.CV_64F)
laplacian = np.uint8(np.absolute(laplacian))

# Canny edge detection
canny = cv2.Canny(gray, 50, 150)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original Grayscale', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(sobelx, cmap='gray')
axes[0, 1].set_title('Sobel X (Vertical edges)', fontsize=12)
axes[0, 1].axis('off')

axes[0, 2].imshow(sobely, cmap='gray')
axes[0, 2].set_title('Sobel Y (Horizontal edges)', fontsize=12)
axes[0, 2].axis('off')

axes[1, 0].imshow(sobel_combined, cmap='gray')
axes[1, 0].set_title('Sobel Combined', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(laplacian, cmap='gray')
axes[1, 1].set_title('Laplacian (2nd derivative)', fontsize=12)
axes[1, 1].axis('off')

axes[1, 2].imshow(canny, cmap='gray')
axes[1, 2].set_title('Canny Edge Detection', fontsize=12)
axes[1, 2].axis('off')

plt.suptitle('Edge Detection Methods', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 2.2 Canny Edge Detection - Multi-Stage

Canny is the most popular edge detector (5 stages)

In [ ]:
# Different threshold values for Canny
canny_low = cv2.Canny(gray, 30, 100)
canny_mid = cv2.Canny(gray, 50, 150)
canny_high = cv2.Canny(gray, 100, 200)

# Overlay edges on original image
overlay = image_rgb.copy()
overlay[canny_mid > 0] = [255, 0, 0]  # Red edges

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(canny_low, cmap='gray')
axes[0, 0].set_title('Canny (30, 100) - More edges', fontsize=11)
axes[0, 0].axis('off')

axes[0, 1].imshow(canny_mid, cmap='gray')
axes[0, 1].set_title('Canny (50, 150) - Balanced', fontsize=11)
axes[0, 1].axis('off')

axes[1, 0].imshow(canny_high, cmap='gray')
axes[1, 0].set_title('Canny (100, 200) - Strong edges only', fontsize=11)
axes[1, 0].axis('off')

axes[1, 1].imshow(overlay)
axes[1, 1].set_title('Edges Overlay on Original', fontsize=11)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("📚 Canny Algorithm Steps:")
print("  1. Gaussian blur (noise reduction)")
print("  2. Gradient calculation (Sobel)")
print("  3. Non-maximum suppression (thin edges)")
print("  4. Double threshold (strong/weak edges)")
print("  5. Edge tracking by hysteresis")

## 2.3 Corner Detection

Corners are important features for object tracking and matching

In [ ]:
# Harris Corner Detection
gray_float = np.float32(gray)
harris = cv2.cornerHarris(gray_float, blockSize=2, ksize=3, k=0.04)
harris = cv2.dilate(harris, None)  # Dilate for visualization

# Shi-Tomasi Corner Detection (Good Features to Track)
corners = cv2.goodFeaturesToTrack(gray, maxCorners=100, qualityLevel=0.01, minDistance=10)

# Visualize
harris_img = image_rgb.copy()
harris_img[harris > 0.01 * harris.max()] = [255, 0, 0]  # Red corners

shi_tomasi_img = image_rgb.copy()
if corners is not None:
    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(shi_tomasi_img, (int(x), int(y)), 3, (0, 255, 0), -1)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(image_rgb)
axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(harris_img)
axes[1].set_title(f'Harris Corners (red)', fontsize=12)
axes[1].axis('off')

axes[2].imshow(shi_tomasi_img)
axes[2].set_title(f'Shi-Tomasi Corners (green) - {len(corners)} found', fontsize=12)
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 2.4 SIFT - Scale-Invariant Feature Transform

SIFT detects keypoints that are invariant to scale and rotation

In [ ]:
# Create SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints_sift, descriptors_sift = sift.detectAndCompute(gray, None)

# Draw keypoints
img_sift = cv2.drawKeypoints(
    image_rgb, keypoints_sift, None,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

plt.figure(figsize=(14, 10))
plt.imshow(img_sift)
plt.title(f'SIFT Keypoints: {len(keypoints_sift)} detected', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"✓ SIFT found {len(keypoints_sift)} keypoints")
print(f"✓ Descriptor shape: {descriptors_sift.shape if descriptors_sift is not None else 'None'}")
print(f"\n📝 Each keypoint has:")
print(f"  - Location (x, y)")
print(f"  - Scale (size)")
print(f"  - Orientation (angle)")
print(f"  - 128-dimensional descriptor vector")

## 2.5 ORB - Oriented FAST and Rotated BRIEF

ORB is a fast, free alternative to SIFT

In [ ]:
# Create ORB detector
orb = cv2.ORB_create(nfeatures=500)

# Detect keypoints and compute descriptors
keypoints_orb, descriptors_orb = orb.detectAndCompute(gray, None)

# Draw keypoints
img_orb = cv2.drawKeypoints(
    image_rgb, keypoints_orb, None,
    color=(0, 255, 0),
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

plt.figure(figsize=(14, 10))
plt.imshow(img_orb)
plt.title(f'ORB Keypoints: {len(keypoints_orb)} detected', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"✓ ORB found {len(keypoints_orb)} keypoints")
print(f"✓ Descriptor shape: {descriptors_orb.shape if descriptors_orb is not None else 'None'}")
print(f"\n⚡ ORB Advantages:")
print(f"  - Much faster than SIFT")
print(f"  - Patent-free")
print(f"  - Binary descriptors (smaller memory)")
print(f"  - Good for real-time applications")

## 2.6 Feature Comparison

Compare all feature detection methods side-by-side

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Original
axes[0, 0].imshow(image_rgb)
axes[0, 0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# Edges
axes[0, 1].imshow(canny, cmap='gray')
axes[0, 1].set_title(f'Canny Edges', fontsize=12)
axes[0, 1].axis('off')

# SIFT
axes[1, 0].imshow(img_sift)
axes[1, 0].set_title(f'SIFT: {len(keypoints_sift)} keypoints', fontsize=12)
axes[1, 0].axis('off')

# ORB
axes[1, 1].imshow(img_orb)
axes[1, 1].set_title(f'ORB: {len(keypoints_orb)} keypoints', fontsize=12)
axes[1, 1].axis('off')

plt.suptitle('Feature Detection Methods Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 2.7 Contour Detection

Find object boundaries in the image

In [ ]:
# Find contours
contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Filter by area
min_area = 500
large_contours = [c for c in contours if cv2.contourArea(c) > min_area]

# Draw contours
contour_img = image_rgb.copy()
cv2.drawContours(contour_img, large_contours, -1, (0, 255, 0), 2)

# Draw bounding boxes
bbox_img = image_rgb.copy()
for contour in large_contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(bbox_img, (x, y), (x+w, y+h), (255, 0, 0), 2)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(contour_img)
axes[0].set_title(f'Contours: {len(large_contours)} objects (area > {min_area})', fontsize=12)
axes[0].axis('off')

axes[1].imshow(bbox_img)
axes[1].set_title('Bounding Boxes', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"Total contours found: {len(contours)}")
print(f"Large contours (area > {min_area}): {len(large_contours)}")

## Summary

**What we learned:**
- ✅ Edge detection: Sobel, Laplacian, Canny
- ✅ Corner detection: Harris, Shi-Tomasi
- ✅ Keypoint detection: SIFT, ORB
- ✅ Contour detection and bounding boxes
- ✅ Feature descriptors for matching

**Next:** Object Detection with YOLO →